# Day 1: Virtual Memory Management Fundamentals

> 🎈 **"What if GPU memory worked like elastic balloons that expand as needed, rather than rigid containers you must size upfront?"**

## Learning Objectives
By the end of this session, you will be able to:
1. Explain the difference between virtual address reservation and physical memory allocation
2. Use VMM APIs to create growable GPU data structures
3. Apply alignment requirements for virtual memory operations
4. Configure memory access permissions for security and correctness

## Why Virtual Memory Management?

Traditional `cudaMalloc`:
- Allocates both virtual address AND physical memory together
- Fixed size - cannot grow without copy
- Simple but inflexible

VMM approach:
- Reserve virtual address range (cheap, can be huge)
- Allocate physical memory separately
- Map physical to virtual on demand
- **Growable data structures without copying!**

## 🎈 Concept Card: Virtual Memory as Elastic Storage Balloons

**The Analogy:** Traditional memory allocation is like buying fixed-size storage boxes—you must guess the right size upfront. VMM is like having **elastic storage balloons**:

```
🎈 ELASTIC BALLOON STORAGE
═══════════════════════════════════════════════════════════
Traditional cudaMalloc:          VMM Approach:
┌─────────────────┐              ┌─────────────────────────┐
│  Fixed Box      │              │  🎈 Elastic Balloon     │
│  ┌───────────┐  │              │                         │
│  │  10 GB    │  │              │  Reserve 100GB space    │
│  │  (wasted  │  │              │  (empty balloon shape)  │
│  │   space!) │  │              │                         │
│  └───────────┘  │              │  Inflate to 1GB → 5GB   │
│                 │              │  → 20GB as needed!      │
│  Can't grow!    │              │                         │
└─────────────────┘              └─────────────────────────┘
        ↓                                  ↓
   Guess wrong →                  Expand smoothly →
   Copy everything!               Zero-copy growth!
═══════════════════════════════════════════════════════════
```

**The Balloon Stages:**
| Stage | Balloon Action | VMM Operation |
|-------|---------------|---------------|
| **Reserve** | Claim balloon shape/space | `cuMemAddressReserve()` - reserve VA range |
| **Allocate** | Buy actual air canisters | `cuMemCreate()` - allocate physical memory |
| **Inflate** | Fill balloon with air | `cuMemMap()` - map physical to virtual |
| **Use** | Open valve for access | `cuMemSetAccess()` - set R/W permissions |
| **Expand** | Add more air when needed | Map more physical to same VA range |

**Why This Matters:** Like a balloon that can expand without changing its location in your room, VMM lets data structures grow without moving—no expensive copy operations!

In [ ]:
%%writefile vmm_basics.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void fillKernel(int* data, int n, int value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = value;
}

__global__ void sumKernel(int* data, int n, long long* result) {
    __shared__ long long sdata[256];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? data[idx] : 0;
    __syncthreads();
    
    for (int s = 128; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    
    if (tid == 0) atomicAdd(result, sdata[0]);
}

int main() {
    // Initialize CUDA Driver API
    cuInit(0);
    
    int device = 0;
    cudaSetDevice(device);
    
    // Step 1: Get allocation properties and granularity
    CUmemAllocationProp prop = {};
    prop.type = CU_MEM_ALLOCATION_TYPE_PINNED;
    prop.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
    prop.location.id = device;
    
    size_t granularity;
    cuMemGetAllocationGranularity(&granularity, &prop,
                                  CU_MEM_ALLOC_GRANULARITY_MINIMUM);
    printf("Allocation granularity: %zu bytes\n", granularity);
    
    // Step 2: Reserve virtual address range (1 GB)
    size_t reserveSize = 1ULL << 30;  // 1 GB
    CUdeviceptr ptr;
    CUresult res = cuMemAddressReserve(&ptr, reserveSize, granularity, 0, 0);
    if (res != CUDA_SUCCESS) {
        printf("Failed to reserve address: %d\n", res);
        return 1;
    }
    printf("Reserved %zu MB virtual address at 0x%llx\n", 
           reserveSize / (1024*1024), (unsigned long long)ptr);
    
    // Step 3: Create physical memory (1 MB initially)
    size_t physSize = 1 << 20;  // 1 MB
    physSize = ((physSize + granularity - 1) / granularity) * granularity;
    
    CUmemGenericAllocationHandle handle;
    res = cuMemCreate(&handle, physSize, &prop, 0);
    if (res != CUDA_SUCCESS) {
        printf("Failed to create physical memory: %d\n", res);
        return 1;
    }
    printf("Created %zu KB physical memory\n", physSize / 1024);
    
    // Step 4: Map physical to virtual
    res = cuMemMap(ptr, physSize, 0, handle, 0);
    if (res != CUDA_SUCCESS) {
        printf("Failed to map memory: %d\n", res);
        return 1;
    }
    printf("Mapped physical to virtual\n");
    
    // Step 5: Set access permissions
    CUmemAccessDesc accessDesc = {};
    accessDesc.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
    accessDesc.location.id = device;
    accessDesc.flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
    
    res = cuMemSetAccess(ptr, physSize, &accessDesc, 1);
    if (res != CUDA_SUCCESS) {
        printf("Failed to set access: %d\n", res);
        return 1;
    }
    printf("Access permissions set\n");
    
    // Use the memory!
    int n = physSize / sizeof(int);
    int* data = (int*)ptr;
    
    fillKernel<<<(n+255)/256, 256>>>(data, n, 1);
    
    long long* d_sum;
    cudaMalloc(&d_sum, sizeof(long long));
    cudaMemset(d_sum, 0, sizeof(long long));
    
    sumKernel<<<(n+255)/256, 256>>>(data, n, d_sum);
    
    long long h_sum;
    cudaMemcpy(&h_sum, d_sum, sizeof(long long), cudaMemcpyDeviceToHost);
    printf("Sum of %d ones = %lld\n", n, h_sum);
    
    // Cleanup
    cudaFree(d_sum);
    cuMemUnmap(ptr, physSize);
    cuMemRelease(handle);
    cuMemAddressFree(ptr, reserveSize);
    
    printf("\nVMM workflow complete!\n");
    return 0;
}

In [ ]:
!nvcc vmm_basics.cu -o vmm_basics -lcuda && ./vmm_basics

## VMM Workflow Summary

```
┌─────────────────────────────────────────────────────┐
│                  Virtual Address Space               │
│  ┌────────────────────────────────────────────────┐ │
│  │ cuMemAddressReserve (1 GB)                     │ │
│  │ ┌──────────┐                                   │ │
│  │ │ Mapped   │ ← cuMemMap                        │ │
│  │ │ (1 MB)   │                                   │ │
│  │ └──────────┘                                   │ │
│  │ [Unmapped space - can grow into later]         │ │
│  └────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────┘
           ↑
           │ cuMemMap
           ↓
┌─────────────────────────────────────────────────────┐
│              Physical Memory (GPU RAM)              │
│  ┌──────────┐                                       │
│  │ cuMemCreate (1 MB)                               │
│  │ (handle)  │                                       │
│  └──────────┘                                       │
└─────────────────────────────────────────────────────┘
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice VMM fundamentals:

In [ ]:
%%writefile vmm_exercises.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

/*
 * VMM Fundamentals Exercises
 * 
 * Exercise 1: Implement a growable array using VMM
 * - Reserve a large virtual address range (e.g., 1GB)
 * - Start with a small physical allocation
 * - Implement grow() that adds more physical memory without copying
 * 
 * Exercise 2: Implement a sparse matrix using VMM
 * - Reserve VA for entire dense matrix
 * - Only allocate physical memory for non-zero blocks
 * 
 * Exercise 3: Measure and compare allocation overhead
 * - Compare cuMemCreate+cuMemMap vs cudaMalloc timing
 */

// Helper macro for CUDA Driver API error checking
#define CU_CHECK(call) do { \
    CUresult err = call; \
    if (err != CUDA_SUCCESS) { \
        const char* errStr; \
        cuGetErrorString(err, &errStr); \
        printf("CUDA Driver Error: %s at %s:%d\n", errStr, __FILE__, __LINE__); \
        exit(1); \
    } \
} while(0)

// Exercise 1: Growable Array Structure
class GrowableVMMArray {
    CUdeviceptr vaBase;      // Virtual address base
    size_t vaSize;           // Total reserved VA size
    size_t physAllocated;    // Currently allocated physical memory
    size_t granularity;      // Allocation granularity
    int device;
    CUmemAllocationProp prop;
    
public:
    GrowableVMMArray(size_t maxSize) : vaSize(maxSize), physAllocated(0) {
        cuInit(0);
        cudaGetDevice(&device);
        
        // Get allocation properties
        memset(&prop, 0, sizeof(prop));
        prop.type = CU_MEM_ALLOCATION_TYPE_PINNED;
        prop.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
        prop.location.id = device;
        
        // Get granularity
        CU_CHECK(cuMemGetAllocationGranularity(&granularity, &prop, 
                                                CU_MEM_ALLOC_GRANULARITY_MINIMUM));
        
        // Round up VA size
        vaSize = ((vaSize + granularity - 1) / granularity) * granularity;
        
        // TODO Exercise 1a: Reserve virtual address range
        // Hint: Use cuMemAddressReserve(&vaBase, vaSize, granularity, 0, 0)
        printf("Exercise 1a: Reserve %zu bytes of virtual address space\n", vaSize);
        
        // YOUR CODE HERE:
        // CU_CHECK(cuMemAddressReserve(...));
    }
    
    bool grow(size_t additionalBytes) {
        // Round up to granularity
        additionalBytes = ((additionalBytes + granularity - 1) / granularity) * granularity;
        
        if (physAllocated + additionalBytes > vaSize) {
            printf("Cannot grow: exceeds reserved VA space\n");
            return false;
        }
        
        // TODO Exercise 1b: Allocate physical memory
        // Hint: cuMemCreate(&handle, additionalBytes, &prop, 0)
        printf("Exercise 1b: Allocate %zu bytes of physical memory\n", additionalBytes);
        
        // YOUR CODE HERE:
        // CUmemGenericAllocationHandle handle;
        // CU_CHECK(cuMemCreate(&handle, additionalBytes, &prop, 0));
        
        // TODO Exercise 1c: Map physical to virtual
        // Hint: cuMemMap(vaBase + physAllocated, additionalBytes, 0, handle, 0)
        printf("Exercise 1c: Map physical memory to virtual address\n");
        
        // YOUR CODE HERE:
        // CU_CHECK(cuMemMap(...));
        
        // TODO Exercise 1d: Set access permissions
        // Hint: cuMemSetAccess with CU_MEM_ACCESS_FLAGS_PROT_READWRITE
        printf("Exercise 1d: Set read/write access permissions\n");
        
        // YOUR CODE HERE:
        // CUmemAccessDesc accessDesc = {};
        // accessDesc.location = prop.location;
        // accessDesc.flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
        // CU_CHECK(cuMemSetAccess(...));
        
        physAllocated += additionalBytes;
        return true;
    }
    
    CUdeviceptr getPtr() const { return vaBase; }
    size_t getAllocatedSize() const { return physAllocated; }
    size_t getReservedSize() const { return vaSize; }
    
    ~GrowableVMMArray() {
        // TODO: Cleanup - unmap, release handle, free VA
        if (vaBase) {
            cuMemUnmap(vaBase, physAllocated);
            cuMemAddressFree(vaBase, vaSize);
        }
    }
};

// Exercise 2: Sparse Matrix Block
struct SparseBlock {
    CUmemGenericAllocationHandle handle;
    bool allocated;
};

// Exercise 3: Timing comparison
void benchmarkAllocationMethods() {
    const size_t SIZE = 1 << 24;  // 16MB
    const int ITERATIONS = 100;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // TODO Exercise 3: Implement timing comparison
    // Compare:
    // 1. cudaMalloc/cudaFree
    // 2. cuMemCreate + cuMemMap + cuMemSetAccess / cuMemUnmap + cuMemRelease
    
    printf("\nExercise 3: Benchmark allocation methods\n");
    printf("Compare cudaMalloc vs VMM allocation for %zu bytes\n", SIZE);
    
    // Method 1: cudaMalloc timing
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        void* ptr;
        cudaMalloc(&ptr, SIZE);
        cudaFree(ptr);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float mallocMs;
    cudaEventElapsedTime(&mallocMs, start, stop);
    printf("cudaMalloc/Free: %.3f ms for %d iterations (%.3f us/alloc)\n", 
           mallocMs, ITERATIONS, mallocMs * 1000 / ITERATIONS);
    
    // YOUR CODE HERE for VMM timing comparison
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    printf("=== VMM Fundamentals Exercises ===\n\n");
    
    // Test Exercise 1: Growable array
    printf("--- Exercise 1: Growable Array ---\n");
    size_t maxSize = 1ULL << 30;  // Reserve 1GB VA space
    GrowableVMMArray growable(maxSize);
    
    // Try growing in steps
    size_t step = 1 << 20;  // 1MB at a time
    for (int i = 0; i < 3; i++) {
        printf("Growing by %zu bytes...\n", step);
        growable.grow(step);
        printf("Current allocated: %zu / %zu bytes\n\n", 
               growable.getAllocatedSize(), growable.getReservedSize());
    }
    
    // Exercise 3: Benchmark
    printf("\n--- Exercise 3: Allocation Benchmark ---\n");
    benchmarkAllocationMethods();
    
    printf("\n=== Exercises Complete ===\n");
    printf("Fill in the TODO sections to complete the exercises!\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o vmm_exercises vmm_exercises.cu -lcuda && ./vmm_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use Python concepts related to VMM. Note that direct VMM access requires CUDA Driver API bindings.

1. **Memory pool simulation**: Create a Python class that simulates VMM behavior using NumPy arrays
2. **Growable buffer pattern**: Implement a buffer that doubles in size using `cuda.to_device()` with copying
3. **Allocation profiling**: Use `cuda.current_context().memory_info()` to track memory usage patterns

## Key Takeaways

### 🎈 Memory Management Patterns Learned

| Pattern | Traditional | VMM Approach | Benefit |
|---------|-------------|--------------|---------|
| **Growable Buffers** | Reallocate + copy | Map more physical | Zero-copy growth |
| **Large Sparse Data** | Waste memory | Reserve VA, map on-demand | Memory efficiency |
| **Dynamic Workloads** | Over-provision | Allocate physical as needed | Cost savings |

### Core VMM Concepts

1. **Virtual ≠ Physical** - Reserve huge VA space (cheap!), allocate small physical memory
2. **Granularity Alignment** - Query and respect allocation granularity requirements
3. **Access Control** - Must explicitly set R/W permissions before kernel access
4. **Growth Without Copy** - Map additional physical memory to reserved VA range

### 🎈 The Elastic Balloon Model
```
Reserve (VA Space) → Allocate (Physical) → Map (Connect) → Access (Permissions)
     🎈 Shape          💨 Air Canisters       🔗 Inflate        🔓 Open Valve
```

**Next:** Stream-ordered allocation for async-friendly memory management!